In [13]:
from behavior_utils import *
import matplotlib.pyplot as plt
import numpy as np

### Initialize paths


In [14]:
root_path = 'S:'
experiment = 'clickbait-motivate'
events_dir = 'bonsai'
session_type = 'm'

# Get paths to relevant files
ttl_paths = get_file_paths(f"{root_path}/{experiment}/preprocessed", 'npy', 'ttl_signal', session_type=session_type, print_paths=True, print_n=np.inf)
video_paths = get_file_paths(f"{root_path}/{experiment}/{events_dir}", 'avi', '', session_type=session_type, print_paths=False, print_n=3)
timestamps = get_file_paths(f"{root_path}/{experiment}/{events_dir}", 'csv', 'timestamp', session_type=session_type, print_paths=False, print_n=3)
event_paths_a = get_file_paths(f"{root_path}/{experiment}/{events_dir}", 'csv', '_eventsA', session_type=session_type, print_paths=False, print_n=3)
event_paths_b = get_file_paths(f"{root_path}/{experiment}/{events_dir}", 'csv', '_eventsB', session_type=session_type, print_paths=False, print_n=3)
sleap_paths = get_file_paths(f"{root_path}/{experiment}/sleap", 'csv', 'sleap', session_type=session_type, print_paths=False, print_n=3)

# Root directory for saving processed data
save_path = Path(f"{root_path}/{experiment}/{events_dir}")

Found 50 ttl_signal.npy files
0 S:\clickbait-motivate\preprocessed\7010\m1\ttl_signal.npy
1 S:\clickbait-motivate\preprocessed\7010\m2\ttl_signal.npy
2 S:\clickbait-motivate\preprocessed\7010\m4\ttl_signal.npy
3 S:\clickbait-motivate\preprocessed\7010\m5\ttl_signal.npy
4 S:\clickbait-motivate\preprocessed\7010\m6\ttl_signal.npy
5 S:\clickbait-motivate\preprocessed\7010\m7\ttl_signal.npy
6 S:\clickbait-motivate\preprocessed\7010\m8\ttl_signal.npy
7 S:\clickbait-motivate\preprocessed\7010\m9\ttl_signal.npy
8 S:\clickbait-motivate\preprocessed\7010\m10\ttl_signal.npy
9 S:\clickbait-motivate\preprocessed\7010\m11\ttl_signal.npy
10 S:\clickbait-motivate\preprocessed\7010\m12\ttl_signal.npy
11 S:\clickbait-motivate\preprocessed\7010\m13\ttl_signal.npy
12 S:\clickbait-motivate\preprocessed\7011\m1\ttl_signal.npy
13 S:\clickbait-motivate\preprocessed\7011\m2\ttl_signal.npy
14 S:\clickbait-motivate\preprocessed\7011\m3\ttl_signal.npy
15 S:\clickbait-motivate\preprocessed\7011\m4\ttl_signal.npy


In [15]:
# Make sure only paths with valid TTL files are used
video_paths = filter_paths(video_paths, ttl_paths)
timestamps = filter_paths(timestamps, ttl_paths)
event_paths_a = filter_paths(event_paths_a, ttl_paths)
event_paths_b = filter_paths(event_paths_b, ttl_paths)
#sleap_paths = filter_paths(sleap_paths, ttl_paths)

print(f"Using {len(video_paths)} .avi files.")
print(f"Using {len(timestamps)} timestamp.csv files.")
print(f"Using {len(event_paths_a)} eventsA.csv files.")
print(f"Using {len(event_paths_b)} eventsB.csv files.")
#print(f"Using {len(sleap_paths)} sleap.csv files.")

Using 52 .avi files.
Using 52 timestamp.csv files.
Using 52 eventsA.csv files.
Using 52 eventsB.csv files.


### Dict of 'names': 'types' for event dataframe columns

In [16]:
columns = {'trial_number': 'uint8',
            'timestamp': 'datetime64[ns]',
            'poke_left': 'bool',
            'poke_right': 'bool',
            'centroid_x': 'uint16',
            'centroid_y': 'uint16',
            'target_cell': 'str',
            'iti': 'bool',
            'water_left': 'bool',
            'water_right': 'bool',
            'reward_state': 'bool',
            'click': 'bool',
            'flip_state': 'bool'  #Uncomment for new sessions (9/26/25)
            }

### Process events and assign TTL times to each observation

In [17]:
print(pd.read_csv(event_paths_a[17]).shape)
print(pd.read_csv(event_paths_b[17]).shape)

(81837, 7)
(81837, 6)


In [18]:
print(event_paths_a[17])
print(event_paths_b[17])

S:\clickbait-motivate\bonsai\7011\m5\10142025_7011_m5_eventsA.csv
S:\clickbait-motivate\bonsai\7011\m5\10142025_7011_m5_eventsB.csv


In [22]:
for session_idx in range(len(video_paths)):
    print(f"{session_idx} {ttl_paths[session_idx]}")
    ttl_times = np.load(ttl_paths[session_idx])
    # Binarize ttl signal and crop to capture portion
    #ttl_full = process_ttl(ttl_paths[session_idx], ttl_floor=0, min_frame_duration=6, min_frame_spacing=6)
    #ttl_crop, ttl_times = crop_ttl(ttl_full)

    event_data = process_events(session_idx, event_paths_a, event_paths_b, columns)
    # Create a column of millisecond timestamps, derived from Bonsai timestamps and offset by the first ttl pulse time
    ttl_offset = len(ttl_times) - len(event_data)
    timestamp_ms =  ttl_times[ttl_offset:]
    event_data['timestamp_ms'] = timestamp_ms.astype(np.int32)

    # add flip_state for 5 minute flip sessions ('flipper' sessions)
    if not 'flip_state' in event_data:
        fps = 30
        minutes = 5
        flip_dur = minutes * 60 * fps
        length = len(event_data)
        vector = (np.arange(length) // flip_dur) % 2
        if session_idx % 2 == 0:
            vector = np.abs(vector-1)
        event_data['flip_state'] = vector

    # Load SLEAP tracking data and concatenate with events df
    if len(sleap_paths) == len(video_paths): 
        sleap_data = pd.read_csv(f"{sleap_paths[session_idx]}")
        sleap_data = clean_sleap(sleap_data)
        sleap_offset = abs(len(sleap_data) - len(event_data))  # Video starts slightly ahead of events
        sleap_data = sleap_data[sleap_offset:].reset_index(drop=True)  # Trim to match event df length
        event_data = pd.concat([event_data, sleap_data], axis=1, join='inner')

    # Set up save path
    mouse_id = ttl_paths[session_idx].parents[1].name
    session_id = ttl_paths[session_idx].parents[0].name
    event_path = save_path / mouse_id / session_id

    # if event path doesn't exist, create it
    if not event_path.exists():
        event_path.mkdir(parents=True, exist_ok=True)
    
    # Save event data
    event_data.to_csv(save_path / mouse_id / session_id / f'events.csv', index=False)
    print(f"Saved events for {mouse_id} {session_id} to {save_path / mouse_id / session_id / f'events.csv'}")

0 S:\clickbait-motivate\preprocessed\7010\m1\ttl_signal.npy
Saved events for 7010 m1 to S:\clickbait-motivate\bonsai\7010\m1\events.csv
1 S:\clickbait-motivate\preprocessed\7010\m2\ttl_signal.npy
Saved events for 7010 m2 to S:\clickbait-motivate\bonsai\7010\m2\events.csv
2 S:\clickbait-motivate\preprocessed\7010\m4\ttl_signal.npy
Saved events for 7010 m4 to S:\clickbait-motivate\bonsai\7010\m4\events.csv
3 S:\clickbait-motivate\preprocessed\7010\m5\ttl_signal.npy
Saved events for 7010 m5 to S:\clickbait-motivate\bonsai\7010\m5\events.csv
4 S:\clickbait-motivate\preprocessed\7010\m6\ttl_signal.npy
Saved events for 7010 m6 to S:\clickbait-motivate\bonsai\7010\m6\events.csv
5 S:\clickbait-motivate\preprocessed\7010\m7\ttl_signal.npy
Saved events for 7010 m7 to S:\clickbait-motivate\bonsai\7010\m7\events.csv
6 S:\clickbait-motivate\preprocessed\7010\m8\ttl_signal.npy
Saved events for 7010 m8 to S:\clickbait-motivate\bonsai\7010\m8\events.csv
7 S:\clickbait-motivate\preprocessed\7010\m9\ttl

IndexError: list index out of range

In [7]:
pd.read_csv()